In [ ]:
print("lets begin the scrapping")

In [2]:
def writetofile(file_path,text_content):
    with open(file_path, 'w',encoding='utf-8') as file:
        file.write(text_content)

In [1]:
# import sys
# !{sys.executable} -m pip install PyPDF2

import requests
from bs4 import BeautifulSoup
import time
import io
# import PyPDF2
import json
import concurrent.futures
import pandas as pd

In [3]:
def city_wise_scrapping(city):

    url = "https://www.pakwheels.com/used-cars/search/-/ct_" + city + "/"
    #url = "https://www.pakwheels.com/used-cars/honda-n-wgn-2016-for-sale-in-lahore-8158521"
    headers = {
        'Accept': '*/*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    html_source = response.text
    soup = BeautifulSoup(html_source, 'html.parser')

    #Find the last page number for that city
    lastpage = int(soup.find('li',class_='last next').find('a').get('href').split('=')[1])
    #print(lastpage)
    alldetails = []

    for i in range(lastpage):
      url = "https://www.pakwheels.com/used-cars/search/-/ct_" + city + "/?page="+ str(i+1)
      #print(url)
      response = requests.get(url, headers=headers)
      html_source = response.text
      soup = BeautifulSoup(html_source, 'html.parser')
      #print(soup.text)
      #Find all 'a' tags with the specified class
      links = soup.find_all('a', class_='car-name ad-detail-path')
      #Extract href attribute from each link
      extracted_links = [link.get('href') for link in links]


      for link in extracted_links:
        try:
          carurl = "https://www.pakwheels.com"+link
          print(carurl)
          response = requests.get(carurl, headers=headers)
          html_source = response.text
          soup = BeautifulSoup(html_source, 'html.parser')

          cardetails={}
          cardetails['title'] = soup.find('div',class_='well').find('h1').text
          cardetails['price'] = soup.find('div',class_='price-box').find('strong').text
          cardetails['sellercontact'] = soup.find('div',class_='ad-numbers pull-right fs12').find('span',class_='generic-green fs16').text
          cardetails['city'] = city
          cardetails['location'] = soup.find('p',class_='detail-sub-heading').find('a').text
          litags = soup.find('ul',class_='gallery light-gallery list-unstyled cS-hidden').find_all('li')
          images = []
          for li in litags:
            images.append(li.get('data-src'))
          cardetails['images'] = images
          cardetails['modelyear'] = soup.find('table',class_='table table-bordered text-center table-engine-detail fs16').find_all('p')[0].text
          cardetails['millage'] = soup.find('table',class_='table table-bordered text-center table-engine-detail fs16').find_all('p')[1].text
          cardetails['fueltype'] = soup.find('table',class_='table table-bordered text-center table-engine-detail fs16').find_all('p')[2].text
          cardetails['transmission'] = soup.find('table',class_='table table-bordered text-center table-engine-detail fs16').find_all('p')[3].text
          scrollcardetails = soup.find('ul',class_='list-unstyled ul-featured clearfix').find_all('li')
          cardetails['registeredin'] = scrollcardetails[1].text
          cardetails['color'] = scrollcardetails[3].text
          cardetails['assembly'] = scrollcardetails[5].text
          cardetails['enginecapacity'] = scrollcardetails[7].text
          cardetails['bodytype'] = scrollcardetails[9].text
          cardetails['lastupdated'] = scrollcardetails[11].text
          cardetails['adref'] = scrollcardetails[13].text
          carfeatures = []
          try:
            features = soup.find('ul',class_='list-unstyled car-feature-list nomargin').find_all('li')
            for feature in features:
              carfeatures.append(feature.text)
          except AttributeError:
            carfeatures.append('not listed')
          cardetails['carfeatures'] = carfeatures
          cardetails['sellercomments'] = soup.find('label', class_='detail-tip show').find_parent().text.replace("Mention PakWheels.com when calling Seller to get a good deal","")

          alldetails.append(cardetails)
        except:
          print("skipped")
          continue


    # Create a DataFrame from the car details
    df = pd.DataFrame(alldetails)
    name = city + '_car_details.csv'

    # Save the DataFrame to an Excel file
    df.to_csv(name, index=False)

    print("Car details saved")

In [4]:
city = "lahore"

# Create a ThreadPoolExecutor with 3 worker threads
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit the tasks to run in parallel
    futures = [executor.submit(city_wise_scrapping, "islamabad") ]
    #futures += [executor.submit(city_wise_scrapping,"islamabad")]
    #futures += [executor.submit(city_wise_scrapping,"lahore")]


    # Wait for all tasks to complete
    concurrent.futures.wait(futures)
print("All Done")

Streaming output truncated to the last 5000 lines.
https://www.pakwheels.com/used-cars/mitsubishi-pajero-1992-for-sale-in-islamabad-8446366
https://www.pakwheels.com/used-cars/toyota-corolla-2016-for-sale-in-islamabad-8446394
https://www.pakwheels.com/used-cars/honda-city-2018-for-sale-in-islamabad-8094320
https://www.pakwheels.com/used-cars/toyota-land-cruiser-2018-for-sale-in-islamabad-8244114
https://www.pakwheels.com/used-cars/honda-city-2016-for-sale-in-islamabad-8446334
https://www.pakwheels.com/used-cars/toyota-aqua-2013-for-sale-in-islamabad-8280795
https://www.pakwheels.com/used-cars/toyota-corolla-2014-for-sale-in-islamabad-8446317
https://www.pakwheels.com/used-cars/suzuki-mehran-2017-for-sale-in-islamabad-8446297
https://www.pakwheels.com/used-cars/suzuki-cultus-2013-for-sale-in-islamabad-8446291
https://www.pakwheels.com/used-cars/prince-pearl-2020-for-sale-in-islamabad-8446268
https://www.pakwheels.com/used-cars/suzuki-alto-2023-for-sale-in-islamabad-8439637
https://www.p

In [1]:
headers = {
        'Accept': '*/*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    }
carurl = "https://www.pakwheels.com/used-cars/toyota-prado-2012-for-sale-in-lahore-7690746"
print(carurl)
response = requests.get(carurl, headers=headers)
html_source = response.text
soup = BeautifulSoup(html_source, 'html.parser')

cardetails={}
cardetails['title'] = soup.find('div',class_='well').find('h1').text
cardetails['location'] = soup.find('p',class_='detail-sub-heading').find('a').text
litags = soup.find('ul',class_='gallery light-gallery list-unstyled cS-hidden').find_all('li')
images = []
for li in litags:
  images.append(li.get('data-src'))
cardetails['images'] = images
cardetails['modelyear'] = soup.find('table',class_='table table-bordered text-center table-engine-detail fs16').find_all('p')[0].text
cardetails['millage'] = soup.find('table',class_='table table-bordered text-center table-engine-detail fs16').find_all('p')[1].text
cardetails['fueltype'] = soup.find('table',class_='table table-bordered text-center table-engine-detail fs16').find_all('p')[2].text
cardetails['transmission'] = soup.find('table',class_='table table-bordered text-center table-engine-detail fs16').find_all('p')[3].text
scrollcardetails = soup.find('ul',class_='list-unstyled ul-featured clearfix').find_all('li')
cardetails['registeredin'] = scrollcardetails[1].text
cardetails['color'] = scrollcardetails[3].text
cardetails['assembly'] = scrollcardetails[5].text
cardetails['enginecapacity'] = scrollcardetails[7].text
cardetails['bodytype'] = scrollcardetails[9].text
cardetails['lastupdated'] = scrollcardetails[11].text
cardetails['adref'] = scrollcardetails[13].text
carfeatures = []
try:
  features = soup.find('ul',class_='list-unstyled car-feature-list nomargin').find_all('li')
  for feature in features:
    carfeatures.append(feature.text)
except AttributeError:
  carfeatures.append('not listed')
cardetails['carfeatures'] = carfeatures
cardetails['sellercomments'] = soup.find('label', class_='detail-tip show').find_parent().text.replace("Mention PakWheels.com when calling Seller to get a good deal","")
print(cardetails)

https://www.pakwheels.com/used-cars/toyota-prado-2012-for-sale-in-lahore-7690746


NameError: name 'requests' is not defined

In [ ]:
def new_car_by_category():

    url = "https://www.pakwheels.com/new-cars/"

    headers = {
        'Accept': '*/*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    html_source = response.text
    soup = BeautifulSoup(html_source, 'html.parser')

    alldetails = []
    extracted_links = []
    #Find all 'a' tags with the specified class for each make/company of cars
    ul = soup.find_all('ul',class_='make-list col-sm-2 list-unstyled new-car-list')

    for each in ul:
      #Extract href attribute from each link
      extracted_links.append(url + each.find('a').get('href').split('/')[2])

    for link in extracted_links:

      response = requests.get(link, headers=headers)
      html_source = response.text
      soup = BeautifulSoup(html_source, 'html.parser')

      #find all new car links of each make/company
      ul = soup.find_all('ul', class_='list-unstyled model-list row item clearfix')[0]
      li = ul.find_all('li')
      for each in li:
        #Extract href attribute from each link
        carurl = link + "/" + each.find('a').get('href').split('/')[3]

        response = requests.get(carurl, headers=headers)
        html_source = response.text
        soup = BeautifulSoup(html_source, 'html.parser')

        cardetails={}
        cardetails['title'] = carurl.split('/')[4] + " " + carurl.split('/')[5]

        #extract all data from table
        try:
          info_table = soup.find('table', class_='bike-version-detailscont')
          info_rows = info_table.find_all('tr')
          # Iterate through table rows
          for row in info_rows:
              columns = row.find_all('td')
              if len(columns) == 2:
                  label = columns[0].text.strip()
                  value = columns[1].text.strip()
                  cardetails[label] = value
        except:
          print('Table not found for ' + cardetails['title'])
          break

        print(cardetails)


        # litags = soup.find('ul',class_='gallery light-gallery list-unstyled cS-hidden').find_all('li')
        # images = []
        # for li in litags:
        #   images.append(li.get('data-src'))
        # cardetails['images'] = images
        alldetails.append(cardetails)

     #Create a DataFrame from the car details
    df = pd.DataFrame(alldetails)
    # Save the DataFrame to an Excel file
    df.to_excel('new_car_details.xlsx', index=False)

    print("Car details saved to 'new_car_details.xlsx'")

In [ ]:
new_car_by_category()